In [6]:
import pandas as pd
import seaborn as sb
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import os

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler



In [8]:
final_data = pd.read_csv("final_data.csv")

In [18]:
final_data.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,Genre
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,hip hop
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,pop
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,reggae


In [ ]:
final_data = pd.DataFrame({})

def getJson(dir):
    with open(dir, 'r') as file:
        data = json.load(file)

    return data

def process_playlist(playlist, playlist_index, playlistCount):
    """Process a single playlist and return its tracks as a DataFrame."""
    local_tracks = []
    trackCount = len(playlist['tracks'])
    for f, track in enumerate(playlist['tracks'], start=1):
        local_tracks.append(track)
        # Note: printing from multiple threads may jumble output.
        #print(f"Progress: {f}/{trackCount} (on playlist {playlist_index}/{playlistCount})")
    return pd.DataFrame(local_tracks)

def getalltracks(data, max_workers=4):
    """Returns a DataFrame containing all tracks from all playlists using multithreading."""
    playlists = data['playlists']
    playlistCount = len(playlists)
    
    # Use ThreadPoolExecutor to process each playlist concurrently
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit a job for each playlist
        futures = [executor.submit(process_playlist, playlist, i, playlistCount)
                   for i, playlist in enumerate(playlists)]
        
        # Collect all the DataFrames as they complete
        df_list = []
        for future in as_completed(futures):
            try:
                df_list.append(future.result())
            except Exception as e:
                print("Error processing a playlist:", e)
    
    # Concatenate all DataFrames from the playlists into one
    retDict = pd.concat(df_list, axis=0, ignore_index=True)
    return retDict

files = os.listdir('Million Playlist')
tracks_list = []

i = 0

for file in files:
    tracks = getalltracks(getJson(f'Million Playlist\{file}'))

    tracks_list.append(tracks)

    i+=1

    print(f"Progress: {i}/{len(files)}")


final_data = pd.concat(tracks_list, axis=0, ignore_index=True)

In [ ]:
ACCESS_TOKEN = "BQDFavo8fE5NhzayKb-MCcj4WoWjTc6emYhjg0FOSCmaaT30uVNglYebrEmTF7tU6SpMLMgEYTvi9B9s1nUuU0PafajR7fgpLnPGqCMf9bb2KD_npAtc4FToG2Clxc9eQXPuoko9tVE"

def get_artist_genre(artist_id):
    """Fetches genres for a given artist ID from Spotify API."""
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        artist_data = response.json()
        return artist_id, artist_data.get("genres", [])  # Return (artist_id, genres)
    elif response.status_code == 429:  # Handle Rate Limiting
        print("Rate limited! Waiting before retrying...")
        time.sleep(5)  # Wait before retrying
        return get_artist_genre(artist_id)  # Retry request
    else:
        print(f"Error fetching artist {artist_id}: {response.status_code}")
        return artist_id, None  # Return None for failed requests
    

rows_with_none_artist_uri = final_data[final_data['Genre'].isna()]['artist_uri'].unique()
artist_ids = [artist.split(":")[2] for artist in rows_with_none_artist_uri]


    
i = 0
    
for artist in artist_ids:
    final_data.loc[final_data['artist_uri'] == artist, 'Genre'] = get_artist_genre(artist)
    i+=1

    print(f"progress: {i}/{len(artist_ids)}")
    

Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...


KeyboardInterrupt: 

In [24]:
ACCESS_TOKEN = "BQDPMSG5-eJEgjp9MN1fXsWrIsVijDFvjmzhAcEVKicOPSJNjf30iWqx1ZfjLqtS5XtdZS0qDDTIEfnBXnkFWI0TmhIK9N1JL_zGjsJgIZ0DLt5_Zsm8yEvXO9ZNUF3Wz2k996pQCEw"
endpoint = "https://api.spotify.com/v1/albums/" # id 

def getReleaseYear(albumId):
    response = requests.get(endpoint + albumID, headers={"Authorization":f"Bearer {ACCESS_TOKEN}"})

    if response.status_code == 200:
        return response.json()['release_date'].split('-')[0]
    elif response.status_code == 429:
        retry = response.headers.get("Retry-After")
        print(f"Rate limited! Waiting {retry}")
        time.sleep(retry)
        return getReleaseYear(albumId)
    elif response.status_code == 404:
        print(f"Failed for {albumID}, content doesn't exist")
        return None
    

    print(response.json())
    response.json()['release_date'].split('-')[0]

albums = final_data[final_data['releaseYear'].isna()]['album_uri'].unique()

i = 0

for album in albums:
    albumID = album.split(":")[2]

    releaseYear = getReleaseYear(albumID)

    if releaseYear:
        final_data.loc[final_data['album_uri'] == album, "releaseYear"] = int(releaseYear)

    i+=1

    print(f"Progress: {i}/{len(albums)}")




Failed for 324ychEJu5FNpV2lI6hLxI, content doesn't exist
Progress: 1/17831
Failed for 4MSG5XbntSyl6VeaMGIYl5, content doesn't exist
Progress: 2/17831
Progress: 3/17831
Progress: 4/17831
Progress: 5/17831
Progress: 6/17831
Progress: 7/17831
Progress: 8/17831
Progress: 9/17831
Progress: 10/17831
Progress: 11/17831
Progress: 12/17831
Progress: 13/17831
Progress: 14/17831
Progress: 15/17831
Progress: 16/17831
Progress: 17/17831
Progress: 18/17831
Progress: 19/17831
Progress: 20/17831
Progress: 21/17831
Progress: 22/17831
Progress: 23/17831
Progress: 24/17831
Progress: 25/17831
Progress: 26/17831
Progress: 27/17831
Progress: 28/17831
Progress: 29/17831
Progress: 30/17831
Progress: 31/17831
Progress: 32/17831
Progress: 33/17831
Progress: 34/17831
Progress: 35/17831
Progress: 36/17831
Progress: 37/17831
Progress: 38/17831
Progress: 39/17831
Progress: 40/17831
Progress: 41/17831
Progress: 42/17831
Progress: 43/17831
Progress: 44/17831
Progress: 45/17831
Progress: 46/17831
Progress: 47/17831
Pr

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
n = len(final_data)
final_data = final_data.iloc[:n//150, ['releaseYear', 'duration_ms']]

fd_scaled = StandardScaler().fit_transform(final_data)

release_years = final_data[(final_data['releaseYear'].isna() == False) & (final_data['releaseYear']>=2000) ]['releaseYear']

dbscan = DBSCAN(eps=.3, min_samples=5)
final_data['cluster'] = dbscan.fit_predict(fd_scaled)

#sb.scatterplot(data=final_data, x=final_data['album_uri'], y=release_years)

ValueError: Found array with 0 sample(s) (shape=(0, 10)) while a minimum of 1 is required by StandardScaler.